# Optimal Computational Budget Allocation in `Spot` {#sec-ocba}

This chapter demonstrates how noisy functions can be handled with Optimal Computational Budget Allocation (OCBA) by `Spot`.

## Example: `Spot`, OCBA,  and the Noisy Sphere Function


In [1]:
import numpy as np
from math import inf
from spotPython.fun.objectivefunctions import analytical
from spotPython.spot import spot
import matplotlib.pyplot as plt
from spotPython.utils.init import fun_control_init
from spotPython.utils.file import get_spot_tensorboard_path
from spotPython.utils.init import fun_control_init, design_control_init, surrogate_control_init

PREFIX = "09"

2024-01-22 13:08:17.246329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 13:08:17.246379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 13:08:17.247760: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 13:08:17.256196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-01-22 13:08:18.498190: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Seed set to 123


### The Objective Function: Noisy Sphere

The `spotPython` package provides several classes of objective functions. We will use an analytical objective function with noise, i.e., a function that can be described by a (closed) formula:
   $$f(x) = x^2 + \epsilon$$

Since `sigma` is set to `0.1`, noise is added to the function:


In [2]:
fun = analytical().fun_sphere
fun_control = fun_control_init(
    PREFIX=PREFIX,
    sigma=0.1)

Seed set to 123


Created spot_tensorboard_path: runs/spot_logs/09_s7-Precision-Tower-7910_2024-01-22_13-08-18 for SummaryWriter()


A plot illustrates the noise:


In [3]:
x = np.linspace(-1,1,100).reshape(-1,1)
y = fun(x, fun_control=fun_control)
plt.figure()
plt.plot(x,y, "k")
plt.show()

<Figure size 1650x1050 with 1 Axes>

`Spot` is adopted as follows to cope with noisy functions:

  1. `fun_repeats` is set to a value larger than 1 (here: 2)
  2. `noise` is set to `true`. Therefore, a nugget (`Lambda`) term is added to the correlation matrix
  3.  `init size` (of the `design_control` dictionary) is set to a value larger than 1 (here: 2)


In [4]:
spot_1_noisy = spot.Spot(fun=fun,
                   fun_control=fun_control_init( 
                   lower = np.array([-1]),
                   upper = np.array([1]),
                   fun_evals = 20,
                   fun_repeats = 2,
                   infill_criterion="ei",
                   noise = True,
                   tolerance_x=0.0,
                   ocba_delta = 1,                   
                   show_models=True),
                   design_control=design_control_init(init_size=3, repeats=2),
                   surrogate_control=surrogate_control_init(noise=True))

Seed set to 123


In [5]:
spot_1_noisy.run()

<Figure size 1650x1050 with 1 Axes>

<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 0.03475493366922229 [####------] 40.00% 


<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 0.00042246988125961625 [#####-----] 50.00% 


...

spotPython tuning: 4.210484834934221e-09 [########--] 80.00% 


<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 2.6768884912567498e-11 [#########-] 90.00% 


<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 2.6768884912567498e-11 [##########] 100.00% Done...



## Print the Results


In [6]:
spot_1_noisy.print_results()

min y: 2.6768884912567498e-11
x0: -5.173865567693801e-06
min mean y: 2.6768884912567498e-11
x0: -5.173865567693801e-06


[['x0', -5.173865567693801e-06], ['x0', -5.173865567693801e-06]]

In [7]:
spot_1_noisy.plot_progress(log_y=False)

<Figure size 2700x1800 with 1 Axes>

## Noise and Surrogates: The Nugget Effect

### The Noisy Sphere

#### The Data

We prepare some data first:


In [8]:
import numpy as np
import spotPython
from spotPython.fun.objectivefunctions import analytical
from spotPython.spot import spot
from spotPython.design.spacefilling import spacefilling
from spotPython.build.kriging import Kriging
import matplotlib.pyplot as plt

gen = spacefilling(1)
rng = np.random.RandomState(1)
lower = np.array([-10])
upper = np.array([10])
fun = analytical().fun_sphere
fun_control = fun_control_init(    
    sigma=2,
    seed=125)
X = gen.scipy_lhd(10, lower=lower, upper = upper)
y = fun(X, fun_control=fun_control)
X_train = X.reshape(-1,1)
y_train = y

Seed set to 125


A surrogate without nugget is fitted to these data:


In [9]:
S = Kriging(name='kriging',
            seed=123,
            log_level=50,
            n_theta=1,
            noise=False)
S.fit(X_train, y_train)

X_axis = np.linspace(start=-13, stop=13, num=1000).reshape(-1, 1)
mean_prediction, std_prediction, ei = S.predict(X_axis, return_val="all")

plt.scatter(X_train, y_train, label="Observations")
plt.plot(X_axis, mean_prediction, label="mue")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Sphere: Gaussian process regression on noisy dataset")

<Figure size 1650x1050 with 1 Axes>

In comparison to the surrogate without nugget, we fit a surrogate with nugget to the data:


In [10]:
S_nug = Kriging(name='kriging',
            seed=123,
            log_level=50,
            n_theta=1,
            noise=True)
S_nug.fit(X_train, y_train)
X_axis = np.linspace(start=-13, stop=13, num=1000).reshape(-1, 1)
mean_prediction, std_prediction, ei = S_nug.predict(X_axis, return_val="all")
plt.scatter(X_train, y_train, label="Observations")
plt.plot(X_axis, mean_prediction, label="mue")
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
_ = plt.title("Sphere: Gaussian process regression with nugget on noisy dataset")

<Figure size 1650x1050 with 1 Axes>

The value of the nugget term can be extracted from the model as follows:


In [11]:
S.Lambda

In [12]:
S_nug.Lambda

7.806881135073833e-05

We see:

  * the first model `S` has no nugget, 
  * whereas the second model has a nugget value (`Lambda`) larger than zero.

## Exercises

### Noisy `fun_cubed`

Analyse the effect of noise on the `fun_cubed` function with the following settings:


In [13]:
fun = analytical().fun_cubed
fun_control = fun_control_init(    
    sigma=10,
    seed=123)
lower = np.array([-10])
upper = np.array([10])

Seed set to 123


### `fun_runge`

Analyse the effect of noise on the `fun_runge` function with the following settings:


In [14]:
lower = np.array([-10])
upper = np.array([10])
fun = analytical().fun_runge
fun_control = fun_control_init(    
    sigma=0.25,
    seed=123)

Seed set to 123


### `fun_forrester`

Analyse the effect of noise on the `fun_forrester` function with the following settings: 


In [15]:
lower = np.array([0])
upper = np.array([1])
fun = analytical().fun_forrester
fun_control = {"sigma": 5,
               "seed": 123}

### `fun_xsin`

Analyse the effect of noise on the `fun_xsin` function with the following settings: 


In [16]:
lower = np.array([-1.])
upper = np.array([1.])
fun = analytical().fun_xsin
fun_control = fun_control_init(    
    sigma=0.5,
    seed=123)

Seed set to 123
